# **CRAWLING DATA DARI WEB BERITA**

## IMPORT MODUL

Modul yang digunakan adalah kombinasi dari tiga modul yaitu import requests
from bs4 import BeautifulSoup
import pandas as pd, yang dapat digunakan untuk mengambil data dari sebuah situs web (menggunakan requests), kemudian mengekstrak informasi yang diinginkan dari HTMLnya (menggunakan BeautifulSoup), dan akhirnya menyimpan hasilnya dalam bentuk dataframe Pandas (menggunakan pd).

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## FUNGSI get_data

In [ ]:
def get_data(url, category):
  data_page = requests.get(url)
  print(url)
  if data_page.status_code == 200:
    data_berita = []
    container = BeautifulSoup(data_page.content, 'html.parser')
    container_page = container.find('div', class_='detail__body-text itp_bodycontent')
    data = container_page.find_all('p')
    judul_berita = container.find('h1', class_='detail__title').text.strip()
    tanggal_berita = container.find('div', class_='detail__date').text.strip()

    data_content = ''.strip()
    for i in data:
      if  'ADVERTISEMENT' not in i.text.strip() and  'SCROLL TO RESUME CONTENT' not in i.text.strip() and i.find('div', class_='detail__multiple') == None:
        data_content += i.text.strip() + ' '

    data_berita.append(judul_berita)
    data_berita.append(tanggal_berita)
    data_berita.append(data_content)
    data_berita.append(category)

    return data_berita

Tujuan dari fungsi ini adalah untuk mengikis data dari halaman web yang ditentukan oleh urlparameter. Berikut rincian kodenya:

1. data_page = requests.get(url): Ini mengirimkan permintaan HTTP GET ke URL yang ditentukan menggunakan requestsperpustakaan dan menyimpan respons dalam data_pagevariabel.

2. print(url): Mencetak URL ke konsol.

3. if data_page.status_code == 200:: Memeriksa apakah kode status respons HTTP adalah 200 (OK), yang menunjukkan permintaan berhasil.

4. Di dalam blok bersyarat:

  - Ini menginisialisasi daftar kosong yang disebut data_berita.
  - Ia menggunakan BeautifulSoup untuk mengurai konten HTML halaman web (data_page.content).
  - Ia menemukan wadah konten berita menggunakan container.find('div', class_='detail__body-text itp_bodycontent').
  - Ini mengekstrak judul berita ( judul_berita) dan tanggal publikasi ( tanggal_berita).
  - Ini menginisialisasi string kosong yang disebut data_content.
  Itu mengulangi semua paragraf (p) dalam wadah dan menambahkan teksnya data_contentjika tidak mengandung kata kunci tertentu seperti 'IKLAN' dan 'GULIR UNTUK MERESUMKAN KONTEN'. Selain itu, ia melewatkan paragraf yang berisi kelas tertentu ('detail__multiple').
  - Itu menambahkan judul, tanggal, konten, dan kategori ke daftar data_berita.
5. Terakhir, ia mengembalikan data_beritadaftar yang berisi informasi yang diekstraksi.

## FUNGSI get_content


In [ ]:
def get_content (url, category):
  menu_page = requests.get(url)
  if menu_page.status_code == 200:
    content = BeautifulSoup(menu_page.content, 'html.parser')
    content_page = content.find('div', class_='grid-row list-content')
    container_content = content_page.find_all('div', class_='media media--left media--image-radius block-link')

    # Untuk menyimpan url content
    data_content = []
    for c in container_content:
      link_content = c.find('a', class_='media__link')
      subtitle = c.find('h2', class_='media__subtitle')
      if subtitle == None and '20.detik' not in str(link_content) and 'intermeso' not in str(link_content) and 'foto' not in str(link_content):
        data_berita = get_data(link_content.get('href'), category)
        data_content.append(data_berita)

    next_page = content.find_all('a', class_='pagination__item')
    for i in next_page:
      if 'Next' == i.text.strip():
        return (data_content, i.get('href'))

Digunakan untuk mengekstrak konten dari URL tertentu yang terkait dengan artikel berita.

- Ini mengirimkan permintaan HTTP GET ke yang ditentukan urlmenggunakan requestsperpustakaan.
- Memeriksa apakah kode status respons adalah 200 (menunjukkan permintaan berhasil).
- Jika permintaan berhasil, permintaan tersebut akan digunakan BeautifulSoupuntuk mengurai konten HTML halaman.
- Menemukan elemen div tertentu dengan kelas 'konten daftar baris kisi' yang kemungkinan berisi konten utama.
- Mengekstrak daftar sub-elemen dengan kelas 'media media--media kiri--tautan blok radius gambar' di dalam konten utama.
- Ulangi sub-elemen ini dan ekstrak informasi seperti tautan ke konten dan subjudul.
- Memeriksa kondisi terkait subtitle dan konten tautan untuk menyaring item yang tidak diinginkan.
- Memanggil get_datafungsi (yang tidak disediakan) untuk mendapatkan data tambahan berdasarkan tautan dan kategori yang diekstraksi.
- Menambahkan data yang diperoleh ke daftar bernama data_content.
- Mencari link penomoran halaman dengan kelas 'pagination__item' untuk memeriksa apakah ada halaman "Berikutnya".
- Jika halaman "Berikutnya" ditemukan, fungsi akan mengembalikan tupel yang berisi daftar data_contentdan URL halaman berikutnya.

## Fungsi scraping_detik_news

In [ ]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 1, category= ['Finance','Sport']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Detik.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

Kode yang diberikan mendefinisikan fungsi scraping_detik_newsyang mengambil data berita dari situs newa.detik.com dan menyimpannya ke file CSV. Berikut rincian fungsionalitas fungsi tersebut:

1. Mengambil Beranda news.detik.com:
  - Fungsi ini mengambil tiga parameter: url, num_content, dan category. urlmenentukan URL beranda Detik.com, num_contentmenunjukkan jumlah artikel berita yang akan dikikis, dan categorymerupakan daftar kategori untuk memfilter artikel berita.
  - Fungsi mengirimkan permintaan GET ke yang ditentukan urlmenggunakan requestsperpustakaan. Jika permintaan berhasil ( response.status_code == 200), konten HTML diambil menggunakan response.content.

2. Mengurai Konten HTML:
  - Konten HTML diurai menggunakan BeautifulSoup menjadi objek sup. Objek sup mewakili struktur HTML halaman web yang diurai.
  - Fungsinya mencari menu navigasi menggunakan find()metode dengan kelas CSS 'static-nav'. Menu navigasi berisi link ke berbagai kategori berita.

3. Mengekstrak Tautan Navbar:

  - Fungsi ini mengulangi tautan yang ditemukan di menu navigasi dan mengekstrak teks tautan dan URL masing-masing menggunakan link.find('div').text.strip()dan link.get('href').
  - Informasi tautan yang diekstraksi disimpan dalam daftar bernama save_link_navbar.

4. Mengikis Artikel Berita:
  - Fungsi ini mengulangi tautan navbar yang diekstraksi dan memfilternya berdasarkan category.
  - Untuk setiap kategori yang dipilih, fungsi ini memanggil get_content()artikel berita untuk kategori tersebut. get_content()mengembalikan daftar artikel berita dan URL halaman berikutnya.
  - Artikel berita yang diambil akan ditambahkan ke data_semua_beritadaftar.

5. Menggores Halaman Berikutnya:
  - Fungsi ini mengekstrak URL halaman berikutnya dari variabel next_page.
  - Ini mengulangi num_content-1waktu, memanggil get_content()untuk mengambil artikel berita dari halaman berikutnya dan menambahkannya ke daftar data_semua_berita.

6. Menyimpan Data yang Tergores:
  - Fungsi ini mengubah data berita yang diambil menjadi Pandas DataFrame.
  - DataFrame disimpan ke file CSV bernama Data Berita Detik.csv.
  - Sebuah pesan dicetak yang menunjukkan keberhasilan penyimpanan data dengan jumlah artikel yang tergores.

Singkatnya, kode ini secara efektif mengambil artikel berita dari Detik.com berdasarkan kategori tertentu dan menyimpan data yang diambil ke file CSV.

## Crawling data

In [ ]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=50, category=['Finance','Sport'])

https://finance.detik.com/moneter/d-7058693/gus-imin-janjikan-khitan-ditanggung-jkn-bpjs-kesehatan-bilang-begini
https://finance.detik.com/bursa-dan-valas/d-7058687/kementerian-bumn-punya-solusi-selamatkan-waskita-yang-mau-didepak-dari-bursa
https://finance.detik.com/berita-ekonomi-bisnis/d-7058627/sedot-sepertiga-uang-negara-pemda-diminta-melek-perubahan-iklim
https://finance.detik.com/berita-ekonomi-bisnis/d-7058633/pertagas-raih-gold-rating-di-ajang-asia-sustainability-reporting-rating
https://finance.detik.com/berita-ekonomi-bisnis/d-7058606/petani-tembakau-jabar-ramai-ramai-tolak-aturan-pengetatan-rokok
https://finance.detik.com/berita-ekonomi-bisnis/d-7058600/tekan-emisi-karbon-ri-nggak-mau-ulang-kesalahan-masa-lalu-ini
https://finance.detik.com/bursa-dan-valas/d-7058591/stafsus-erick-ungkap-divestasi-saham-vale-kemahalan-nggak-masuk-hitungan
https://finance.detik.com/berita-ekonomi-bisnis/d-7058590/mendag-lapor-jokowi-mau-atur-perdagangan-tanaman-kratom
https://finance.detik.com


Fungsi dari kode scraping_detik_news() adalah untuk mengambil data berita dari situs news.Detik.com berdasarkan kategori yang ditentukan. Data berita yang diambil meliputi judul, tanggal, isi berita, dan kategori. Data berita kemudian disimpan ke dalam file CSV dengan nama Data Berita Detik.csv.

## Memanggil data hasil crawling

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PPW/TUGAS/Data Berita newsdetik.csv')
df

,Judul,Tanggal,Artikel,Category
0,"Gus Imin Janjikan Khitan Ditanggung JKN, BPJS ...","Senin, 27 Nov 2023 13:44 WIB",Cawapres nomor urut 1 Muhaimin Iskandar atau G...,Finance
1,Kementerian BUMN Punya Solusi Selamatkan Waski...,"Senin, 27 Nov 2023 13:43 WIB",Saham PT Waskita Karya (Persero) Tbk terancam ...,Finance
2,"Sedot Sepertiga Uang Negara, Pemda Diminta Mel...","Senin, 27 Nov 2023 13:33 WIB","Menteri Keuangan, Sri Mulyani Indrawati, menje...",Finance
3,Pertagas Raih Gold Rating di Ajang Asia Sustai...,"Senin, 27 Nov 2023 13:31 WIB",PT Pertamina Gas (Pertagas) yang merupakan bag...,Finance
4,Petani Tembakau Jabar Ramai-ramai Tolak Aturan...,"Senin, 27 Nov 2023 13:28 WIB",Ratusan petani dan komunitas pertembakauan di ...,Finance
...,...,...,...,...
1617,"Seandainya Ada Team Order dari Ducati, Jorge M...","Senin, 09 Okt 2023 20:10 WIB",Persaingan ketat di antara Francesco Bagnaia d...,Sport
1618,"Yamaha Mau VR46 Racing Jadi Tim Satelit, tapi...","Senin, 09 Okt 2023 19:30 WIB",Yamaha ingin mempunyai tim satelit lagi musim ...,Sport
1619,Tim Kempo Indonesia Sabet 4 Emas di Kejuaraan ...,"Senin, 09 Okt 2023 18:50 WIB",Tim Kempo Indonesia bersinar di Negeri Sakura ...,Sport
1620,Jadwal MotoGP Mandalika Akhir Pekan Ini,"Senin, 09 Okt 2023 18:15 WIB",Jadwal MotoGP Mandalika akhir pekan ini bisa d...,Sport


In [ ]:
df.shape

(1622, 4)

In [ ]:
df.isnull().sum()

Judul       0
Tanggal     0
Artikel     0
Category    0
dtype: int64